In [1]:
from datetime import datetime
from glob import glob
import pandas as pd
import xarray as xr
import json
import os

In [2]:
PATH_RESOURCES = os.path.join('..', '..', 'resources')
PATH_LOCATIONS = os.path.join(PATH_RESOURCES, 'utils', 'locations-webcam.json')
PATH_DATA      = os.path.join(PATH_RESOURCES, 'reanalysis-era5-single-levels')

In [3]:
def createDataFrameVars():
    list_var_path       = glob(os.path.join(PATH_DATA, '*', ''))
    dic_var_all_paths  = list()

    for var_path in list_var_path:
        for var_file in glob(os.path.join(var_path, '*.netcdf')):
            path_var_basename = os.path.basename(var_file)
            year  = int(path_var_basename.split('_')[-2])
            month = int(path_var_basename.split('_')[-3])
            date  = datetime(year, month, 1)

            dic_var_all_paths.append({
                'variable' : var_path.split(os.sep)[-2],
                'date_time': date.strftime('%Y-%m'),
                'path'    : var_file
            })

    return pd.DataFrame(dic_var_all_paths)

In [4]:
def createDataFrameLocs():
    with open(PATH_LOCATIONS, 'r') as file:
        df = pd.DataFrame(json.load(file)['locations'])

    return df

In [5]:
df_locs = createDataFrameLocs()
df_vars = createDataFrameVars()

In [6]:
df_locs

,name,id,lat,lon
0,Catedral de Burgos,cburgos,42.33998,-3.70373
1,El Pobo â€“ vista desde Mirador Hoyalta,pobo,40.53797,-0.91138


In [7]:
df_vars

,variable,date_time,path
0,cloud_base_height,2024-01,..\..\resources\reanalysis-era5-single-levels\...
1,cloud_base_height,2024-02,..\..\resources\reanalysis-era5-single-levels\...
2,cloud_base_height,2024-03,..\..\resources\reanalysis-era5-single-levels\...
3,cloud_base_height,2024-04,..\..\resources\reanalysis-era5-single-levels\...
4,high_cloud_cover,2024-01,..\..\resources\reanalysis-era5-single-levels\...
5,high_cloud_cover,2024-02,..\..\resources\reanalysis-era5-single-levels\...
6,high_cloud_cover,2024-03,..\..\resources\reanalysis-era5-single-levels\...
7,high_cloud_cover,2024-04,..\..\resources\reanalysis-era5-single-levels\...
8,low_cloud_cover,2024-01,..\..\resources\reanalysis-era5-single-levels\...
9,low_cloud_cover,2024-02,..\..\resources\reanalysis-era5-single-levels\...
